In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


#Loading the dataset
dataset = pd.read_csv("dataset(clean).csv")
#number of sameples in dataset
print("Total number of sample: "+str(len(dataset)))

Total number of sample: 916575


In [2]:
dataset.Emotion.value_counts()

disappointed    313714
happy           301871
angry           300990
Name: Emotion, dtype: int64

In [3]:
dataset.head(10)

,Emotion,Content,Original Content
0,disappointed,oh fuck did i wrote fil grinningfacewithsweat ...,b'RT @Davbingodav: @mcrackins Oh fuck.... did ...
1,disappointed,i feel nor am i shamed by it,i feel nor am i shamed by it
2,disappointed,i had been feeling a little bit defeated by th...,i had been feeling a little bit defeated by th...
3,happy,imagine if that reaction guy that called jj kf...,"b""@KSIOlajidebt imagine if that reaction guy t..."
4,disappointed,i wouldnt feel burdened so that i would live m...,i wouldnt feel burdened so that i would live m...
5,angry,who ha time for that it s super sacrilegious,"b""@V_actually Who has time for that?? It's SUP..."
6,happy,lol facewithtearsofjoy facewithtearsofjoy face...,b'@BriMatjuda @drtlaleng Lol:face_with_tears_o...
7,angry,i hope i get the job cause im in desperate nee...,i hope i get the job cause im in desperate nee...
8,happy,awhh he cute lmao im on the first episode face...,"b'@KimDionysus182 Awhhh he cute, lmao Im on th..."
9,disappointed,the true problem with leftist social space,b'@EmSC_00 @PrincessOtC @petercoffin The true ...


In [4]:
input_sentences = [text.split(" ") for text in dataset["Content"].values.tolist()]
labels = dataset["Emotion"].values.tolist()

In [5]:
# Initialize word2id and label2id dictionaries that will be used to encode words and labels
word2id = dict()
label2id = dict()

max_words = 0 # maximum number of words in a sentence

# Construction of word2id dict
for sentence in input_sentences:
    for word in sentence:
        # Add words to word2id dict if not exist
        if word not in word2id:
            word2id[word] = len(word2id)
    # If length of the sentence is greater than max_words, update max_words
    if len(sentence) > max_words:
        max_words = len(sentence)

    
# Construction of label2id and id2label dicts
label2id = {l: i for i, l in enumerate(set(labels))}
id2label = {v: k for k, v in label2id.items()}
print(label2id)
print(max_words)
print(len(word2id))

{'happy': 0, 'disappointed': 1, 'angry': 2}
178
168965


In [6]:
import keras
from sklearn.model_selection import train_test_split
# Encode input words and labels
X = [[word2id[word] for word in sentence] for sentence in input_sentences]
Y = [label2id[label] for label in labels]
# Apply Padding to X
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, max_words)

# Convert Y to numpy array
Y = keras.utils.to_categorical(Y, num_classes=len(label2id), dtype='float32')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
# Print shapes
print("Shape of X_train: {}".format(X_train.shape))
print("Shape of Y_train: {}".format(Y_train.shape))
print("Shape of X_test: {}".format(X_test.shape))
print("Shape of Y_test: {}".format(Y_test.shape))


Using TensorFlow backend.


Shape of X_train: (733260, 178)
Shape of Y_train: (733260, 3)
Shape of X_test: (183315, 178)
Shape of Y_test: (183315, 3)


In [7]:
embedding_dim = 21 # Calculated by the rule of thumb

# Define input tensor
sequence_input = keras.Input(shape=(max_words,), dtype='int32')

# Word embedding layer
embedded_inputs =keras.layers.Embedding(len(word2id) + 1,
                                        embedding_dim,
                                        input_length=max_words)(sequence_input)

# Apply dropout to prevent overfitting
embedded_inputs = keras.layers.Dropout(0.2)(embedded_inputs)

# Apply Bidirectional LSTM over embedded inputs
lstm_outs = keras.layers.wrappers.Bidirectional(
    keras.layers.LSTM(embedding_dim, return_sequences=True)
)(embedded_inputs)

# Apply dropout to LSTM outputs to prevent overfitting
lstm_outs = keras.layers.Dropout(0.2)(lstm_outs)

# Attention Mechanism - Generate attention vectors
input_dim = int(lstm_outs.shape[2])
permuted_inputs = keras.layers.Permute((2, 1))(lstm_outs)
attention_vector = keras.layers.TimeDistributed(keras.layers.Dense(1))(lstm_outs)
attention_vector = keras.layers.Reshape((max_words,))(attention_vector)
attention_vector = keras.layers.Activation('softmax', name='attention_vec')(attention_vector)
attention_output = keras.layers.Dot(axes=1)([lstm_outs, attention_vector])

# Last layer: fully connected with softmax activation
fc = keras.layers.Dense(embedding_dim, activation='relu')(attention_output)
output = keras.layers.Dense(len(label2id), activation='softmax')(fc)

# Finally building model
model = keras.Model(inputs=[sequence_input], outputs=output)
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer='adam')

# Print model summary
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 178)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 178, 21)      3548286     input_1[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 178, 21)      0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 178, 42)      7224        dropout_1[0][0]                  
____________________________________________________________________________________________

In [8]:
import tensorflow as tf
my_callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max',patience=2),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
]
model.fit(X, Y, epochs=20, batch_size=64, validation_split=0.2, shuffle=True, callbacks=my_callbacks)

C:\Users\kkswe\OneDrive\Desktop\Anaconda\lib\site-packages\tensorflow\python\framework\indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 733260 samples, validate on 183315 samples
Epoch 1/20
733260/733260 [==============================] - 1368s 2ms/step - loss: 0.2934 - accuracy: 0.8722 - val_loss: 0.2128 - val_accuracy: 0.9093
Epoch 2/20
733260/733260 [==============================] - 1350s 2ms/step - loss: 0.2011 - accuracy: 0.9149 - val_loss: 0.2085 - val_accuracy: 0.9101
Epoch 3/20
733260/733260 [==============================] - 1333s 2ms/step - loss: 0.1839 - accuracy: 0.9231 - val_loss: 0.2101 - val_accuracy: 0.9096
Epoch 4/20
733260/733260 [==============================] - 1329s 2ms/step - loss: 0.1717 - accuracy: 0.9286 - val_loss: 0.2075 - val_accuracy: 0.9101
Epoch 5/20
733260/733260 [==============================] - 1403s 2ms/step - loss: 0.1625 - accuracy: 0.9326 - val_loss: 0.2135 - val_accuracy: 0.9105
Epoch 6/20
733260/733260 [==============================] - 1313s 2ms/step - loss: 0.1548 - accuracy: 0.9357 - val_loss: 0.2174 - val_accuracy: 0.9089
Epoch 7/20
733260/733260 [================

In [9]:
# Re-create the model to get attention vectors as well as label prediction
model_with_attentions = keras.Model(inputs=model.input,
                                    outputs=[model.output, 
                                             model.get_layer('attention_vec').output])

In [10]:
results = model.evaluate(X_test, Y_test, batch_size=64)
print('test loss, test acc:', results)

183315/183315 [==============================] - 36s 198us/step
test loss, test acc: [0.1437685579405353, 0.9402285814285278]


In [61]:
from sklearn.metrics import classification_report, confusion_matrix
target_names=['disappointed','angry','happy']
y_pred=model.predict(X_test)
print(y_pred.shape)
print(Y.shape)
print(classification_report(Y_test.argmax(axis=1)
                            ,y_pred.argmax(axis=1),
                            target_names=target_names))

(183315, 3)
(233, 3, 3, 3, 3)
              precision    recall  f1-score   support

disappointed       0.99      0.96      0.98     60683
       angry       0.92      0.93      0.93     62547
       happy       0.91      0.93      0.92     60085

    accuracy                           0.94    183315
   macro avg       0.94      0.94      0.94    183315
weighted avg       0.94      0.94      0.94    183315

